In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 5.1 MB/s 


In [ ]:
acids={
    'A': ['GCA', 'GCC', 'GCG', 'GCT'],
    'N': ['AAC', 'AAT'],
    'C': ['TGC', 'TGT'],
    'D': ['GAC', 'GAT'],
    'E': ['GAA', 'GAG'],
    'F': ['TTC', 'TTT'],
    'G': ['GGA', 'GGC', 'GGG', 'GGT'],
    'H': ['CAC', 'CAT'],
    'I': ['ATA', 'ATC', 'ATT'],
    'K': ['AAA', 'AAG'],
    'L': ['CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'],
    'M': ['ATG'],
    'N': ['AAC', 'AAT'],
    'P': ['CCA', 'CCC', 'CCG', 'CCT'],
    'Q': ['CAA', 'CAG'],
    'R': ['AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGT'],
    'S': ['AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT'],
    'T': ['ACA', 'ACC', 'ACG', 'ACT'],
    'V': ['GTA', 'GTC', 'GTG', 'GTT'],
    'W': ['TGG'],
    'Y': ['TAC', 'TAT'],
    'Stop': ['TAA', 'TAG', 'TGA']
 
}

In [50]:
#check existence of a file
import os
name_of_file=input("Please enter a name of file with genome sequence with the extention:\t")
os.path.isfile(name_of_file)

Please enter a name of file with genome sequence with the extention:	genome.txt


True

In [ ]:
#import genome sequence from a file
import os
from Bio.Restriction import *
from Bio.Seq import Seq
from pathlib import Path
if (os.path.isfile(name_of_file)):
    seqe = Path(name_of_file).read_text() 
    # get rid of \n
    seqe = seqe.replace('\n', '')

In [135]:
#Checking if inputted enzyme exists 

from Bio.Restriction import AllEnzymes, Restriction

def input_enzyme_name():
  input_message = 'Please write name of an enzyme (0 to stop):'
  given_enzyme_name_is_correct = False
  enz_list=[]
  enz_name=input(input_message)
  while not given_enzyme_name_is_correct:
    if enz_name=='0':
      given_enzyme_name_is_correct=True
      return list(set(enz_list))
    else:
      try:
        enz=eval(enz_name)
        
        enz_list.append(enz)
        print('Given name is correct')
        # where the enzyme is
        print(f'Your enzyme is {enz} which cuts at {enz.site} and the restriction site is in {enz.search(seq)} place in your genome sequence')
        enz_name=input(input_message)
      except NameError:
        print('Given name is incorrect, please try again:')
        enz_name=input(input_message)
  
  return RestrictionBatch(set(enz_list))



In [122]:
def all_positions(genome,enzymes): #find restriction sites of all enzymes that you specify
    restr_sites={}
    for enz in enzymes:
      restr_sites[enz]=enz.search(genome)
    return restr_sites

In [70]:
def find_coding_regions(myseq):
    s1=myseq #frame 1
    s2=myseq[1:] #frame 2
    s3=myseq[2:] #frame 3
    frames=[s1,s2,s3] 
    seq_translated=[] #dna frames to be translated
    
    for seq in frames: #iterate over each frame
        if len(seq)%3==1: #add N to avoid errors
            seq=seq+'NN'
            seq_translated.append(seq.translate())
        elif len(seq)%3==2:
            seq=seq+'N'
            seq_translated.append(seq.translate())
        else:
            seq_translated.append(seq.translate())

    pos=[[],[]] #positions of coding regions: first list is start positions, second
              #list is stop positions
    for transl in seq_translated: #find the first instance of START
        start=transl.find('M')
        
        if start==-1: #if no starting codons, terminate
            return pos
        else:
            while start!=-1: #iterate until run out of starting codons
                stop=transl.find('*') #find STOP codons
                if stop!=-1:  
                    pos[0].append(start) #add coordinates
                    pos[1].append(stop) 
                    start=transl[stop:].find('M') #new search is in the frame after the
                    #coding region
                else:
                    return pos
                    
                
            if start==-1:
                return pos
    return pos

In [67]:
def find_dna(myseq,pos): #find the dna sequences of coding regions
    coding_regions=[]
    for ind in range(len(pos[0])): 
        start=pos[0][ind]
        stop=pos[1][ind]
        coding_regions.append(myseq[start*3:stop*3+3]) #dna sequence that codes for protein
    return coding_regions

In [130]:
def silent_mutation(coding_region, coordinates, restr_sites,acids):
    coding_region_start=coordinates[0] #coordinates is made of two lists: starting points of Coding sequences 
    coding_region_stop=coordinates[1] #and stopping points 

    for enz in restr_sites: #iterate over each enzyme
        restr_start_sites=restr_sites[enz] #create a list of cutting positions
        i=0
        if restr_start_sites!=[]: #pick only those that actually cut our sequence
            while i !=len(restr_start_sites): #iterate through all cutting points
                start_site= restr_start_sites[i]-coding_region_start-1 #we shift by 1 nucleotide and also
                                    #by the coordinate of coding sequence, because coding sequence is a 
                                #sliced version of genome (so when we slice a sequence that is in the
                #middle of genome, we adjust the cutting point according to the position of coding sequence)
                if start_site>=0 and start_site<=coding_region_stop-3: #restriction site is in the coding region
                    f1=(start_site-1)%3 #check which frame the cutting point is
                    f2=(start_site-2)%3
                    f3=(start_site-3)%3
                    if f1==0:
                        translation_start=start_site-1 #shift 1 left
                    elif f2==0: 
                        translation_start=start_site-2 #shift 2 left
                    elif f3==0:
                        translation_start=start_site #no shift = correct frame
                    restr_site_codon=coding_region[translation_start:translation_start+3] #pick the cutting point and flanking nucleotides
                    restr_site_acid=restr_site_codon.translate() #translate region that contains restriction site
                    codons=acids[restr_site_acid].copy()
                    codons.remove(restr_site_codon)
                    if codons!=[]:
                        coding_region=coding_region.replace(restr_site_codon,codons[0])
                
                    else:
                        break
                i+=1
    return coding_region

In [ ]:
if __name__ == '__main__':
  genome=Seq(seqe.upper()) #convert all characters to uppercase
  enz=input_enzyme_name()
  positions=all_positions(genome, enz) #all cutting positions
  coord=find_coding_regions(genome) #coding sequences and their coordinates
  if coord!=[]:
    cds=find_dna(genome,coord)
    for i in range(len(cds)):
        new_genome=silent_mutation(coding_region=cds[i], coordinates=[coords[0][i],coords[1][i]],restr_sites=positions,acids=acids)
        print('original genome:',genome,'\nmutated genome',new_genome)
  else:
    print("no coding regions")